<a href="https://colab.research.google.com/github/vhrique/anne_ptbr/blob/development/04_Arquiteturas_Especializadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Introdução à Arquiteturas Especializadas

As arquiteturas clássicas de redes neurais, como as redes densas (MLPs), convolucionais (CNNs) e recorrentes (RNNs), são amplamente usadas para tarefas de classificação e regressão, aproveitando sua estrutura para extrair padrões dos dados e realizar previsões. Essas redes, em geral, aprendem a mapear entradas para saídas rotuladas e são versáteis em diferentes domínios.

No entanto, há arquiteturas especializadas que atendem a demandas mais específicas. Autoencoders, por exemplo, são redes projetadas para compressão de dados e extração de características latentes, enquanto GANs (Redes Generativas Adversárias) são usadas para a geração de dados sintéticos realistas. Em tarefas de visão computacional, arquiteturas voltadas para detecção de objetos e segmentação aprendem a identificar e localizar múltiplas classes em uma imagem, demonstrando a evolução do campo para resolver problemas além da classificação tradicional.

Nesta aula, falaremos mais sobre estes tipos de arquiteturas e como isto muda em relação à representação, avaliação e otimização de modelos.

# Autoencoders

Os autoencoders são um tipo de rede neural projetada para aprender uma representação compacta dos dados, também conhecida como codificação latente, de forma não supervisionada. Eles são compostos por duas partes principais: o codificador, que reduz a dimensionalidade dos dados de entrada ao projetá-los em um espaço latente de menor dimensão, e o decodificador, que tenta reconstruir os dados originais a partir dessa representação comprimida. A principal aplicação dos autoencoders está em tarefas como redução de dimensionalidade, remoção de ruído e geração de novas amostras.

A avaliação de autoencoders geralmente é feita comparando a similaridade entre os dados de entrada e sua reconstrução gerada pelo decodificador, utilizando métricas como erro quadrático médio (MSE) ou entropia cruzada, dependendo do tipo de dados. O objetivo é minimizar essa diferença, o que indica que o autoencoder aprendeu uma boa representação latente. A otimização dos autoencoders é realizada através de técnicas padrão de redes neurais, como a _backpropagation_ e o uso de algoritmos de otimização, como o gradiente descendente e suas variantes.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, input_size)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
model = Autoencoder(input_size=1_000)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

## Variational Autoencoders

Os autoencoders variacionais (VAEs) são uma extensão dos autoencoders tradicionais que introduzem uma abordagem probabilística para a geração de dados. Em vez de codificar os dados diretamente em um espaço latente fixo, os VAEs aprendem distribuições probabilísticas que representam as características latentes dos dados, com a rede aprendendo os parâmetros de média e variância dessas distribuições. Essa estrutura permite que o modelo gere novas amostras ao amostrar dessas distribuições latentes, o que torna os VAEs ideais para tarefas como síntese de dados, geração de imagens e aprendizado de características latentes com maior capacidade de generalização.

Em termos de avaliação e otimização, além da reconstrução dos dados, o VAE incorpora na função de perda um termo de regularização chamado divergência Kullback-Leibler (KL), que mede o quão próxima a distribuição latente aprendida está de uma distribuição normal padrão. A otimização, então, busca equilibrar dois objetivos: minimizar a perda de reconstrução, para garantir que a entrada seja bem reconstruída, e minimizar a divergência KL, para manter a estrutura latente bem regularizada. Esse processo permite que o VAE não apenas reconstrua os dados, mas também gere novas amostras de maneira mais controlada e com maior capacidade de generalização.

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_size):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 400),
            nn.ReLU()
        )
        self.mu_layer = nn.Linear(400, 20)
        self.logvar_layer = nn.Linear(400, 20)  # Log variância

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(20, 400),
            nn.ReLU(),
            nn.Linear(400, input_size),
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        encoded = self.encoder(x)
        mu = self.mu_layer(encoded)
        logvar = self.logvar_layer(encoded)
        z = self.reparameterize(mu, logvar)
        decoded = self.decoder(z)
        return decoded, mu, logvar

def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) # KL Diverg.
    return BCE + KLD

In [ ]:
vae = VAE(input_size=1_000)
optimizer = optim.Adam(vae.parameters(), lr=0.001)

# Visão Computacional

## Detecção de Objetos

### R-CNN (2014)

### Fast R-CNN (2015)

### Faster R-CNN (2015)

### YOLO (2015 - ...)

### RetinaNet (2017)

### DETR (2020)

## Segmentação de Imagens

### FCN (2015)

### U-Net (2015)

### DeepLab (2015 - ...)

### Mask R-CNN (2017)

### SAM (2023)

## Geração e Transformação de Imagens

### Generative Adversarial Networks

### StyleGAN


### CycleGAN

### PixelRNN

# Processamento de Linguagem Natural

## Seq2Seq